In [30]:
import importlib
import pandas as pd
from datetime import datetime
import numpy as np

from bs4 import BeautifulSoup
from src import scrapers, helpers
importlib.reload(scrapers)
importlib.reload(helpers)
test_scraper = scrapers.PfrScraper(
    '/Users/Jared/OneDrive - Sportsbook Science/data/raw/nfl')

In [160]:
test_scraper.request_season_summaries(range(1970, 1980))

1970 Season Summary already saved locally, skipping
1971 Season Summary already saved locally, skipping
1972 Season Summary already saved locally, skipping
1973 Season Summary already saved locally, skipping
1974 Season Summary already saved locally, skipping
1975 Season Summary already saved locally, skipping
1976 Season Summary already saved locally, skipping
1977 Season Summary already saved locally, skipping
1978 Season Summary already saved locally, skipping
1979 Season Summary already saved locally, skipping


In [2]:
sum_html = test_scraper.load_season_summary(1993)
sum_df = test_scraper.scrape_season_summary(sum_html)
print(sum_df['week_summary_links'][0].split(','))
print(sum_df['week_labels'][0].split(','))
sum_df.head()


['/years/1993/week_1.htm', '/years/1993/week_2.htm', '/years/1993/week_3.htm', '/years/1993/week_4.htm', '/years/1993/week_5.htm', '/years/1993/week_6.htm', '/years/1993/week_7.htm', '/years/1993/week_8.htm', '/years/1993/week_9.htm', '/years/1993/week_10.htm', '/years/1993/week_11.htm', '/years/1993/week_12.htm', '/years/1993/week_13.htm', '/years/1993/week_14.htm', '/years/1993/week_15.htm', '/years/1993/week_16.htm', '/years/1993/week_17.htm', '/years/1993/week_18.htm', '/years/1993/week_19.htm', '/years/1993/week_20.htm', '/years/1993/week_21.htm', '/years/1993/week_22.htm']
['Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Week 7', 'Week 8', 'Week 9', 'Week 10', 'Week 11', 'Week 12', 'Week 13', 'Week 14', 'Week 15', 'Week 16', 'Week 17', 'Week 18', 'Wild Card', 'Divisional', 'Conf Champ', 'Super Bowl']


,season,num_weeks,week_summary_links,week_labels
0,1993,22,"/years/1993/week_1.htm,/years/1993/week_2.htm,...","Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week..."


In [5]:
week_sum_html = test_scraper.load_week_summary(season=2021, week=2)
week_sum_df = test_scraper.scrape_week_summary(week_sum_html)
week_sum_df.head()


,num_games,game_summary_links
0,16,"/boxscores/202109160was.htm,/boxscores/2021091..."


In [6]:
game_links = week_sum_df['game_summary_links'][0].split(',')
game_html = test_scraper.get_with_requests(test_scraper.base_url+game_links[0])



In [33]:
game_summary_df = pd.DataFrame({
    'pfr_link': ['/boxscores/202109160was.htm'],
    'week_no': [1],
    'week_label': ['Week 1']
})

game_summary_df['pfr_game_id'] = game_summary_df['pfr_link'][0].split('/')[2].split('.')[0]
game_summary_df['is_playoff_game'] = ~game_summary_df['week_label'].str.contains('Week')
# game_summary_df['game_start_datetime_est'] = datetime()



game_soup = BeautifulSoup(game_html, 'html.parser')

## Scorebox Parsing ##
scorebox_text_list = [x.get_text() for x in game_soup.find(class_='scorebox_meta').find_all('div')]

# Datetime info
date_of_game = scorebox_text_list[0]
date_of_game_split = date_of_game.split(' ')
day_of_week = date_of_game_split[0]
game_summary_df['day_of_week'] = day_of_week

month_str = date_of_game_split[1]
month_str_full = helpers.month_abrv_lkp(month_str)
day_num = date_of_game_split[2][:-1]
year = date_of_game_split[3]

# Check if Datetime info is in scorebox, if not, return np.nan
exists, start_time = helpers.check_if_startswith(
    'Start Time: ', scorebox_text_list)
if exists:
    hh_mm = start_time[:-2]
    ampm = start_time[-2:].upper()
    date_str = f"{day_num}-{month_str}-{year} {hh_mm} {ampm}"
    dt_obj = datetime.strptime(date_str, "%d-%b-%Y %I:%M %p")
    game_summary_df['game_start_datetime'] = dt_obj
else:
    game_summary_df['game_start_datetime'] = np.nan

# Check stadium (if exists)

# Game Summary Info
# Get Linescore
linescore = str(
    game_soup.find('table', class_='linescore nohover stats_table no_freeze'))
linescore_df = pd.read_html(linescore)[0]

# Check Overtime
if 'OT' in linescore_df.columns:
    game_summary_df['overtime_played'] = True
    ot_score_home = linescore_df['OT'][1]
    ot_score_away = linescore_df['OT'][0]
else:
    game_summary_df['overtime_played'] = False
    ot_score_home = 0
    ot_score_away = 0

game_summary_df['home_team'] = linescore_df['Unnamed: 1'][1]
game_summary_df['home_team_score_q1'] = linescore_df['1'][1]
game_summary_df['home_team_score_q2'] = linescore_df['2'][1]
game_summary_df['home_team_score_q3'] = linescore_df['3'][1]
game_summary_df['home_team_score_q4'] = linescore_df['4'][1]
game_summary_df['home_team_score_overtime'] = ot_score_home
game_summary_df['home_team_score_final'] = linescore_df['Final'][1]
game_summary_df['away_team'] = linescore_df['Unnamed: 1'][0]
game_summary_df['away_team_score_q1'] = linescore_df['1'][0]
game_summary_df['away_team_score_q2'] = linescore_df['2'][0]
game_summary_df['away_team_score_q3'] = linescore_df['3'][0]
game_summary_df['away_team_score_q4'] = linescore_df['4'][0]
game_summary_df['away_team_score_overtime'] = ot_score_away
game_summary_df['away_team_score_final'] = linescore_df['Final'][0]


print(linescore_df.head())
game_summary_df.head()



                         Unnamed: 0                Unnamed: 1  1   2   3   4  \
0  via Sports Logos.net About logos           New York Giants  7   3  10   9   
1  via Sports Logos.net About logos  Washington Football Team  0  14   3  13   

   Final  
0     29  
1     30  


,pfr_link,week_no,week_label,pfr_game_id,is_playoff_game,day_of_week,game_start_datetime,overtime_played,home_team,home_team_score_q1,...,home_team_score_q4,home_team_score_overtime,home_team_score_final,away_team,away_team_score_q1,away_team_score_q2,away_team_score_q3,away_team_score_q4,away_team_score_overtime,away_team_score_final
0,/boxscores/202109160was.htm,1,Week 1,202109160was,False,Thursday,2021-09-16 20:20:00,False,Washington Football Team,0,...,13,0,30,New York Giants,7,3,10,9,0,29


In [31]:
print(scorebox_text_list)
scorebox_text_list[0].rindex('Thursday')
helpers.check_if_startswith(sub_str='Thursday', text_list=scorebox_text_list)


['Thursday Sep 16, 2021', 'Start Time: 8:20pm', 'Stadium: FedExField ', 'Attendance: 50,118', 'Time of Game: 3:19', 'Logos via Sports Logos.net\n            / About logos']


(True, 'Sep 16, 2021')